In [173]:
import pandas as pd
import numpy as np

df = pd.read_csv('Annotations.csv')
df.drop(['url', 'URL(*)', 'Q2.11 Other media', 'Q2.12 News presented type', 'Q2.13 Visual bias'], axis=1, inplace=True)
df.fillna('', inplace=True)


# Pulizie generali

df = df.applymap(lambda x: x.rstrip(';') if isinstance(x, str) else x)
df


/tmp/ipykernel_11054/1533059379.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)
/tmp/ipykernel_11054/1533059379.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.rstrip(';') if isinstance(x, str) else x)


,id,title,label,annotator,Q.1.1 Type of news,Q.1.2 Accuracy,Q.1.3 Headline,Q1.3.1 Quotation,Q.1.4 Fact-based lead,Q.1.5 Bias,Q.1.6 Sensational language,Q1.7 Negative target,"Q1.7.1 If the article negatively targets a specific group, please check the types of targeting from the options below",Q1.7.1.2 Please specify the type,Q2.8 Attribution,Q2.9.1 Date,Q2.9.2 Exact Date,Q2.10 event 30 days
0,18d71db7-f362-46fa-953e-d8353c7f7232,Coronavirus| nuove regole: al tavolo al ristor...,Group 2,Manuel,1.0,2.0,3,,2,4.0,4.0,1,,,1.0,2.0,31-05-2021,2
1,18d71db7-f362-46fa-953e-d8353c7f7232,Coronavirus| nuove regole: al tavolo al ristor...,Group 2,Marinella,,,,,,,,,,,,,,
2,18d71db7-f362-46fa-953e-d8353c7f7232,Coronavirus| nuove regole: al tavolo al ristor...,Group 2,Angelo,1.0,3.0,1;3,1,1,4.0,4.0,1,,,1.0,2.0,31-05-2021,2
3,3ca70f03-38f3-4a7b-bdd3-f012b22e0c7e,Israele: Lapid| ostacoli ma fiducioso su nuovo...,Group 2,Manuel,1.0,4.0,1;3,,2,4.0,4.0,1,,,3.0,2.0,31-05-2021,2
4,3ca70f03-38f3-4a7b-bdd3-f012b22e0c7e,Israele: Lapid| ostacoli ma fiducioso su nuovo...,Group 2,Marinella,1.0,4.0,1;3,-,1,4.0,4.0,2,11,,3.0,2.0,31-05-2021,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058,5b30e244-4aa2-4ad3-b24a-2022c1415df7,Cosa fare dei documenti della Stasi,Group 1,Marinella,3.0,3.0,1;8,-,1,4.0,4.0,1,,,1.0,2.0,17-06-2021,2
1059,5b30e244-4aa2-4ad3-b24a-2022c1415df8,Cosa fare dei documenti della Stasi,Group 1,Angelo,3.0,3.0,1;9,,2,4.0,4.0,1,,,1.0,2.0,17-06-2021,2
1060,eab45a62-1d47-4216-a51d-e45ccbcc3810,"Strani dolori allo stomaco, dopo 6 mesi muore ...",Group 1,Manuel,,,,,,,,,,,,,,
1061,eab45a62-1d47-4216-a51d-e45ccbcc3810,"Strani dolori allo stomaco, dopo 6 mesi muore ...",Group 1,Marinella,1.0,4.0,1;9,-,1,4.0,3.0,1,,,3.0,2.0,25-06-2021,2


In [174]:
# Rinomina le colonne utilizzando solo i numeri
df = df.rename(columns={
    'Q.1.1 Type of news': 'Q1.1',
    'Q.1.2 Accuracy': 'Q1.2',
    'Q.1.3 Headline': 'Q1.3',
    'Q1.3.1 Quotation': 'Q1.3.1',
    'Q.1.4 Fact-based lead': 'Q1.4',
    'Q.1.5 Bias': 'Q1.5',
    'Q.1.6 Sensational language': 'Q1.6',
    'Q1.7 Negative target': 'Q1.7',
    'Q1.7.1 If the article negatively targets a specific group, please check the types of targeting from the options below': 'Q1.7.1',
    'Q1.7.1.2 Please specify the type': 'Q1.7.1.2',
    'Q2.8 Attribution': 'Q2.8',
    'Q2.9.1 Date': 'Q2.9.1',
    'Q2.9.2 Exact Date': 'Q2.9.2',
    'Q2.10 event 30 days': 'Q2.10'
})

df

,id,title,label,annotator,Q1.1,Q1.2,Q1.3,Q1.3.1,Q1.4,Q1.5,Q1.6,Q1.7,Q1.7.1,Q1.7.1.2,Q2.8,Q2.9.1,Q2.9.2,Q2.10
0,18d71db7-f362-46fa-953e-d8353c7f7232,Coronavirus| nuove regole: al tavolo al ristor...,Group 2,Manuel,1.0,2.0,3,,2,4.0,4.0,1,,,1.0,2.0,31-05-2021,2
1,18d71db7-f362-46fa-953e-d8353c7f7232,Coronavirus| nuove regole: al tavolo al ristor...,Group 2,Marinella,,,,,,,,,,,,,,
2,18d71db7-f362-46fa-953e-d8353c7f7232,Coronavirus| nuove regole: al tavolo al ristor...,Group 2,Angelo,1.0,3.0,1;3,1,1,4.0,4.0,1,,,1.0,2.0,31-05-2021,2
3,3ca70f03-38f3-4a7b-bdd3-f012b22e0c7e,Israele: Lapid| ostacoli ma fiducioso su nuovo...,Group 2,Manuel,1.0,4.0,1;3,,2,4.0,4.0,1,,,3.0,2.0,31-05-2021,2
4,3ca70f03-38f3-4a7b-bdd3-f012b22e0c7e,Israele: Lapid| ostacoli ma fiducioso su nuovo...,Group 2,Marinella,1.0,4.0,1;3,-,1,4.0,4.0,2,11,,3.0,2.0,31-05-2021,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058,5b30e244-4aa2-4ad3-b24a-2022c1415df7,Cosa fare dei documenti della Stasi,Group 1,Marinella,3.0,3.0,1;8,-,1,4.0,4.0,1,,,1.0,2.0,17-06-2021,2
1059,5b30e244-4aa2-4ad3-b24a-2022c1415df8,Cosa fare dei documenti della Stasi,Group 1,Angelo,3.0,3.0,1;9,,2,4.0,4.0,1,,,1.0,2.0,17-06-2021,2
1060,eab45a62-1d47-4216-a51d-e45ccbcc3810,"Strani dolori allo stomaco, dopo 6 mesi muore ...",Group 1,Manuel,,,,,,,,,,,,,,
1061,eab45a62-1d47-4216-a51d-e45ccbcc3810,"Strani dolori allo stomaco, dopo 6 mesi muore ...",Group 1,Marinella,1.0,4.0,1;9,-,1,4.0,3.0,1,,,3.0,2.0,25-06-2021,2


In [175]:
def Q1_1(df, column):
    df[column] = df[column].astype(str)
    df[column] = df[column].str.replace('.0', '')
    valid_values = ['1', '2', '3', '4', '5', '0']
    df[column] = df[column].apply(lambda x: x if x in valid_values else '0')
    invalid_rows = df[~df[column].isin(valid_values)].index
    for i in invalid_rows:
        print(f"Invalid value in row {i} for column '{column}': {df.loc[i, column]}")

def Q1_2(df, column):
    df[column] = df[column].astype(str)
    df[column] = df[column].str.replace('.0', '')
    df[column] = df[column].str.replace('nan', '')
    valid_values = ['1', '2', '3', '4', '0']
    df[column] = df[column].apply(lambda x: x if x in valid_values else '0')
    invalid_rows = df[df[column].isin(['']) | ~df[column].isin(valid_values)].index
    for i in invalid_rows:
        print(f"Invalid value in row {i} for column '{column}': {df.loc[i, column]}")

def Q1_3(df, column):
    df[column] = df[column].astype(str)
    df[column] = df[column].apply(lambda x: '' if not x.replace(';', '').isdigit() else x)

def Q1_31(df, column):
    df[column] = df[column].astype(str)
    valid_values = ['1', '2', '0']
    df[column] = df[column].apply(lambda x: x if x in valid_values else '0')
    invalid_rows = df[~df[column].isin(valid_values)].index
    for i in invalid_rows:
        print(f"Invalid value in row {i} for column '{column}': {df.loc[i, column]}")

def Q1_4(df, column):
    df[column] = df[column].astype(str)
    valid_values = ['1', '2', '0']
    df[column] = df[column].apply(lambda x: x if x in valid_values else '0')
    invalid_rows = df[~df[column].isin(valid_values)].index
    for i in invalid_rows:
        print(f"Invalid value in row {i} for column '{column}': {df.loc[i, column]}")

def Q1_5(df, column):
    df[column] = df[column].astype(str)
    df[column] = df[column].str.replace('.0', '')
    valid_values = ['1', '2', '3', '4', '0']
    df[column] = df[column].apply(lambda x: x if x in valid_values else '0')
    invalid_rows = df[~df[column].isin(valid_values)].index
    for i in invalid_rows:
        print(f"Invalid value in row {i} for column '{column}': {df.loc[i, column]}")

def Q1_6(df, column):
    df[column] = df[column].astype(str)
    df[column] = df[column].str.replace('.0', '')
    valid_values = ['1', '2', '3', '4', '0']
    df[column] = df[column].apply(lambda x: x if x in valid_values else '0')
    invalid_rows = df[~df[column].isin(valid_values)].index
    for i in invalid_rows:
        print(f"Invalid value in row {i} for column '{column}': {df.loc[i, column]}")

def Q1_7(df, column):
    df[column] = df[column].astype(str)
    valid_values = ['1', '2', '0']
    df[column] = df[column].apply(lambda x: x if x in valid_values else '0')
    invalid_rows = df[~df[column].isin(valid_values)].index
    for i in invalid_rows:
        print(f"Invalid value in row {i} for column '{column}': {df.loc[i, column]}")

def Q1_71(df, column):
    df[column] = df[column].astype(str)
    df[column] = df[column].apply(lambda x: '' if not x.replace(';', '').isdigit() else x)


def Q2_8(df, column):
    df[column] = df[column].astype(str)
    valid_values = ['1', '2', '3', '0']
    df[column] = df[column].apply(lambda x: x if x in valid_values else '0')
    invalid_rows = df[~df[column].isin(valid_values)].index
    for i in invalid_rows:
        print(f"Invalid value in row {i} for column '{column}': {df.loc[i, column]}")

def Q2_91(df, column):
    df[column] = df[column].astype(str)
    valid_values = ['1', '2', '0']
    df[column] = df[column].apply(lambda x: x if x in valid_values else '0')
    invalid_rows = df[~df[column].isin(valid_values)].index
    for i in invalid_rows:
        print(f"Invalid value in row {i} for column '{column}': {df.loc[i, column]}")

def Q2_92(df, column):
    df[column] = df[column].astype(str)
    df[column] = pd.to_datetime(df[column], format='%d-%m-%Y')

def Q2_10(df, column):
    df[column] = df[column].astype(str)
    valid_values = ['1', '2', '0']
    df[column] = df[column].apply(lambda x: x if x in valid_values else '0')
    invalid_rows = df[~df[column].isin(valid_values)].index
    for i in invalid_rows:
        print(f"Invalid value in row {i} for column '{column}': {df.loc[i, column]}")

Q1_1(df, 'Q1.1')

Q1_2(df, 'Q1.2')

Q1_3(df, 'Q1.3')

Q1_31(df, 'Q1.3.1')

Q1_4(df, 'Q1.4')

Q1_5(df, 'Q1.5')

Q1_6(df, 'Q1.6')

Q1_7(df, 'Q1.7')

Q1_71(df, 'Q1.7.1')

Q2_8(df, 'Q2.8')

Q2_91(df, 'Q2.9.1')

Q2_92(df, 'Q2.9.2')

Q2_10(df, 'Q2.10')

In [176]:
df

,id,title,label,annotator,Q1.1,Q1.2,Q1.3,Q1.3.1,Q1.4,Q1.5,Q1.6,Q1.7,Q1.7.1,Q1.7.1.2,Q2.8,Q2.9.1,Q2.9.2,Q2.10
0,18d71db7-f362-46fa-953e-d8353c7f7232,Coronavirus| nuove regole: al tavolo al ristor...,Group 2,Manuel,1,2,3,0,2,4,4,1,,,0,0,2021-05-31,2
1,18d71db7-f362-46fa-953e-d8353c7f7232,Coronavirus| nuove regole: al tavolo al ristor...,Group 2,Marinella,0,0,,0,0,0,0,0,,,0,0,NaT,0
2,18d71db7-f362-46fa-953e-d8353c7f7232,Coronavirus| nuove regole: al tavolo al ristor...,Group 2,Angelo,1,3,1;3,1,1,4,4,1,,,0,0,2021-05-31,2
3,3ca70f03-38f3-4a7b-bdd3-f012b22e0c7e,Israele: Lapid| ostacoli ma fiducioso su nuovo...,Group 2,Manuel,1,4,1;3,0,2,4,4,1,,,0,0,2021-05-31,2
4,3ca70f03-38f3-4a7b-bdd3-f012b22e0c7e,Israele: Lapid| ostacoli ma fiducioso su nuovo...,Group 2,Marinella,1,4,1;3,0,1,4,4,2,11,,0,0,2021-05-31,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058,5b30e244-4aa2-4ad3-b24a-2022c1415df7,Cosa fare dei documenti della Stasi,Group 1,Marinella,3,3,1;8,0,1,4,4,1,,,0,0,2021-06-17,2
1059,5b30e244-4aa2-4ad3-b24a-2022c1415df8,Cosa fare dei documenti della Stasi,Group 1,Angelo,3,3,1;9,0,2,4,4,1,,,0,0,2021-06-17,2
1060,eab45a62-1d47-4216-a51d-e45ccbcc3810,"Strani dolori allo stomaco, dopo 6 mesi muore ...",Group 1,Manuel,0,0,,0,0,0,0,0,,,0,0,NaT,0
1061,eab45a62-1d47-4216-a51d-e45ccbcc3810,"Strani dolori allo stomaco, dopo 6 mesi muore ...",Group 1,Marinella,1,4,1;9,0,1,4,3,1,,,0,0,2021-06-25,2


In [177]:
# CASTO I TIPI E PULISCO

# Columns to convert to integers
columns_to_convert = ['Q1.1', 'Q1.2', 'Q1.3.1', 'Q1.4', 'Q1.5', 'Q1.6', 'Q1.7', 'Q2.8', 'Q2.9.1', 'Q2.10']

for col in columns_to_convert:
    df[col] = df[col].fillna(0).astype(int)
    # Sostituisci tutte le stringhe vuote con 0 in tutto il dataframe
    df[col].replace(r'^\s*$', '0', regex=True, inplace=True)



df['Q1.3'] = df['Q1.3'].str.split(';')
df['Q1.7.1'] = df['Q1.7.1'].str.split(';')


# # Sostituire i NaN con 0 nella colonna 'Q.1.3 Headline'
# df['Q1.3'].fillna(0, inplace=True)

# # Sostituire i NaN con 0 nella colonna 'Q.1.7.1'
# df['Q1.7.1'].fillna(0, inplace=True)


df = df.query('`Q1.1` != 0 & `Q1.2` != 0')


df

/tmp/ipykernel_11054/3205679644.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].replace(r'^\s*$', '0', regex=True, inplace=True)
/tmp/ipykernel_11054/3205679644.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tr

,id,title,label,annotator,Q1.1,Q1.2,Q1.3,Q1.3.1,Q1.4,Q1.5,Q1.6,Q1.7,Q1.7.1,Q1.7.1.2,Q2.8,Q2.9.1,Q2.9.2,Q2.10
0,18d71db7-f362-46fa-953e-d8353c7f7232,Coronavirus| nuove regole: al tavolo al ristor...,Group 2,Manuel,1,2,[3],0,2,4,4,1,[],,0,0,2021-05-31,2
2,18d71db7-f362-46fa-953e-d8353c7f7232,Coronavirus| nuove regole: al tavolo al ristor...,Group 2,Angelo,1,3,"[1, 3]",1,1,4,4,1,[],,0,0,2021-05-31,2
3,3ca70f03-38f3-4a7b-bdd3-f012b22e0c7e,Israele: Lapid| ostacoli ma fiducioso su nuovo...,Group 2,Manuel,1,4,"[1, 3]",0,2,4,4,1,[],,0,0,2021-05-31,2
4,3ca70f03-38f3-4a7b-bdd3-f012b22e0c7e,Israele: Lapid| ostacoli ma fiducioso su nuovo...,Group 2,Marinella,1,4,"[1, 3]",0,1,4,4,2,[11],,0,0,2021-05-31,2
6,34b92f4c-32db-454c-a369-ca2032443f6c,Le borsette fatte a 3 euro l'ora vendute a 820...,Group 1,Manuel,2,3,[3],0,1,3,2,2,[11],Fashion craft industries system,0,0,NaT,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056,ec3a0585-9da6-4732-bf58-c6e659d8e58e,La grande ondata di caldo nel Nord America,Group 1,Angelo,1,3,"[1, 9]",1,2,3,4,1,[],,0,0,2021-06-30,2
1058,5b30e244-4aa2-4ad3-b24a-2022c1415df7,Cosa fare dei documenti della Stasi,Group 1,Marinella,3,3,"[1, 8]",0,1,4,4,1,[],,0,0,2021-06-17,2
1059,5b30e244-4aa2-4ad3-b24a-2022c1415df8,Cosa fare dei documenti della Stasi,Group 1,Angelo,3,3,"[1, 9]",0,2,4,4,1,[],,0,0,2021-06-17,2
1061,eab45a62-1d47-4216-a51d-e45ccbcc3810,"Strani dolori allo stomaco, dopo 6 mesi muore ...",Group 1,Marinella,1,4,"[1, 9]",0,1,4,3,1,[],,0,0,2021-06-25,2


In [178]:
df.dtypes


id                   object
title                object
label                object
annotator            object
Q1.1                  int64
Q1.2                  int64
Q1.3                 object
Q1.3.1                int64
Q1.4                  int64
Q1.5                  int64
Q1.6                  int64
Q1.7                  int64
Q1.7.1               object
Q1.7.1.2             object
Q2.8                  int64
Q2.9.1                int64
Q2.9.2       datetime64[ns]
Q2.10                 int64
dtype: object

In [179]:
df.to_csv('Annotations_checked.csv', encoding='utf-8', index=False)